---
**Execute this tutorial in Binder** [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/MolSSI/zenopy/5a28d6babe6d197bfd3725f0b207065789fc0a9a?urlpath=lab%2Ftree%2Fdocs%2Fnotebooks%2Fquick-start.ipynb)

---

This guide demonstrates how you can use ``zenopy`` to search through Zenodo's public records and
retrieve individual records as well. For more tutorials, see the [documentation](https://molssi.github.io/zenopy).

---
**Note**

The following section, **Create a config file**, is not required to be able to perform
search operations through the Zenodo public records. However, Zenodo 
[recommends](https://developers.zenodo.org/#authentication) all API requests be 
authenticated and over HTTPS.

---

### **Create a config file**

When you want to use ``zenopy`` for the first time, you need to give it access to your Zenodo (Sandbox) account. The first step is to store your authentication token(s) in a config file. You can create a new text file (say, **.zenodorc**) in the current directory and store our token in it as shown below

In [ ]:
%%writefile ./.zenodorc
[ZENODO]
token = 

[SANDBOX]
token = 

---
**Note**

If you do not know how to create an authentication token, refer to the [documentation](https://molssi.github.io/zenopy/howtos/client/cli_token.html).

---

The config file you just created is going to be read by the ``zenopy`` client object to allow you access your personal Zenodo depositions.
In addition, the created token will authenticate your API requests sent to the Zenodo server.

### **Initialize the client**

The next step is to import the ``zenopy`` package

In [1]:
import zenopy

Then, call the ``zenopy``'s client constructor and tell it where the config file is

In [2]:
cli = zenopy.Zenodo(token="", use_sandbox=False)

---
**Note**

Note that because we plan to use ``zenopy`` only for searching through Zenodo's 
public (and published) records, we can create the client object without an
authentication token (empty string) as follows

```python
    zenopy.Zenodo(token="", use_sandbox=False)
```

We also set the ``use_sandbox`` argument to ``False`` because we want to 
search Zenodo's public servers (not those of Zenodo Sandbox).

---

Great. Now, the ``zenopy`` client is connected to your Zenodo account server and is ready to interact with it.

### **Initialize a _Records object**

The client object allows us to create an instance of the ``_Records`` class

In [3]:
rec_obj = cli.init_records()
rec_obj

### **Search Through Zenodo’s Public Records**

Now, you can use the **depo_obj** to create a new deposition draft in your Zenodo account and get a handle to it

In [6]:
rec_list = rec_obj.list_records()
rec_list

The value of 'content_type' argument is None.
ZenoPy will adopt JSON encoding.

The value of 'status' argument is None.
ZenoPy will search for 'published' record.

The value of 'sort' argument is None.
ZenoPy will sort the search results according to 'bestmatch' sort option.



In [12]:
def print_record_titles(record_list: list[zenopy.record.Record] = None) -> None:
    if record_list is None:
        raise RuntimeError("The 'record_list' argument cannot be None.")
    for record_i in record_list:
        print(record_i.title)

Nice. Let's use the deposition handle (**my_depo**) to check its content via its ``data`` attribute

In [13]:
print_record_titles(record_list=rec_list)

Attempted hybridization between Oryza sativa L. and Porteresia coarctata T
Histological observation of callus morphology in rice
DNA content in rice hybrids and heterosis
Factors affecting pollen embryogenesis of rice anther culture
Genetic studies on purple pigmentation in rice and its use in breeding two-line rice hybrids
Calorific values of 60 rice varieties
Cibodas, a high-yielding variety with good grain quality
Screening of basmati rice genotypes against blast
Preliminary evidence of aphid resistance to transgenic plants
Role of host plant resistance in successful control of brown planthopper in central Luzon, Philippines


In [14]:
molssi_rec_list = rec_obj.list_records(communities="molssi")

The value of 'content_type' argument is None.
ZenoPy will adopt JSON encoding.

The value of 'status' argument is None.
ZenoPy will search for 'published' record.

The value of 'sort' argument is None.
ZenoPy will sort the search results according to 'bestmatch' sort option.



In [15]:
print_record_titles(molssi_rec_list)

SEAMM: Simulation Environment for Atomistic and Molecular Modeling
Plug-in for SEAMM for building fluid systems with PACKMOL.
MolSSI Guidelines on APOD Cyclic Parallelization Strategy
Plug-in for SEAMM to allow custom python scripts in flowcharts
DES5M
DESS66 and DESS66x8
DES370K
DES15K
Plug-in for SEAMM to create structures from SMILES
MolSSI Formatting Guidelines for Machine Learning Products


Upon construction, each deposition is provided with a unique DOI and an ID. Let's print them out here